In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import pandas as pd
import time
import random
import imaplib
import email
import re

# ----------------- Email Sending Script -----------------

# Load the Excel file containing recipient information
file_path = "Names_Final.xlsx"  # Replace with the actual path to your Excel file
df = pd.read_excel(file_path)

# Sender accounts (Replace with environment variables or a secure secrets manager)
sender_accounts = [
    {"email": "your_email_1@example.com", "password": "your_app_password_1"},
    {"email": "your_email_2@example.com", "password": "your_app_password_2"},
    {"email": "your_email_3@example.com", "password": "your_app_password_3"},
    {"email": "your_email_4@example.com", "password": "your_app_password_4"}
]

# Email subject (Replace with an appropriate subject when using)
subject = "Your Email Subject Here"

# Email body template (Replace with actual email content)
email_body_template = """
Hi {creator_name},

[Insert your email content here]

Best regards,  
[Your Team Name]
"""

# Loop through each row in the Excel file and send emails
for index, row in df.iterrows():
    receiver_email = row['Email IDs']  # Fetch email ID
    creator_name = row['Creator Name']  # Fetch creator name

    # Rotate sender accounts cyclically to prevent spam filtering
    account_index = index % len(sender_accounts)
    sender_email = sender_accounts[account_index]['email']
    password = sender_accounts[account_index]['password']

    # Create personalized email body
    body = email_body_template.format(creator_name=creator_name)

    # Create the email message
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'html'))

    try:
        # Connect to SMTP server
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()  # Secure the connection
        server.login(sender_email, password)

        # Send the email
        server.sendmail(sender_email, receiver_email, msg.as_string())
        print(f"Email sent to {creator_name} at {receiver_email} from {sender_email}")

        # Close the server connection
        server.quit()

    except Exception as e:
        print(f"Failed to send email to {creator_name} at {receiver_email}. Error: {e}")

    # Add a random delay between emails to avoid spam detection
    delay_per_email = random.uniform(90, 150)
    print(f"Waiting for {delay_per_email:.1f} seconds before sending the next email...")
    time.sleep(delay_per_email)

print("All emails sent successfully!")

# ----------------- Email Reply Handling -----------------

# Load the second Excel file for matching email responses
merged_handles_df = pd.read_excel('List_Final.xlsx')  # Ensure correct file path

# Add a new column to store extracted phone numbers
merged_handles_df['PhoneNumber'] = '-'

for account in sender_accounts:
    EMAIL = account['email']
    PASSWORD = account['password']

    # Connect to IMAP server to check email responses
    mail = imaplib.IMAP4_SSL('imap.gmail.com', 993)
    mail.login(EMAIL, PASSWORD)
    mail.select("inbox")

    # Search for replies related to the sent campaign
    status, messages = mail.search(None, '(SUBJECT "Re: Your Email Subject Here")')

    email_ids = messages[0].split()

    for email_id in email_ids:
        # Fetch email content
        status, msg_data = mail.fetch(email_id, "(RFC822)")
        for response in msg_data:
            if isinstance(response, tuple):
                msg = email.message_from_bytes(response[1])

                # Extract sender's email address
                from_email = msg.get("From")
                match = re.search(r'<(.*?)>', from_email)  # Extract email inside <>
                email_address = match.group(1) if match else from_email

                # Extract phone number from email body
                email_body = ""
                if msg.is_multipart():
                    for part in msg.walk():
                        content_type = part.get_content_type()
                        if content_type == "text/plain":
                            email_body = part.get_payload(decode=True).decode()
                            break
                else:
                    email_body = msg.get_payload(decode=True).decode()

                # Extract 10-digit phone numbers
                phone_numbers = re.findall(r'(\+91[-\s]?[6-9]\d{9}|[6-9]\d{9})', email_body)
                phone_number = phone_numbers[0] if phone_numbers else "Not Provided"

                # Update phone number in the dataset if email matches
                if email_address in merged_handles_df['Email IDs'].values:
                    merged_handles_df.loc[merged_handles_df['Email IDs'] == email_address, 'PhoneNumber'] = phone_number

    # Logout from the email account
    mail.logout()

# Save the updated Excel file with extracted phone numbers
merged_handles_df.to_excel("List_Updated_WithNumbers.xlsx", index=False, engine='openpyxl')

print("Replies processed and phone numbers updated successfully!")

# ----------------- Follow-Up Emails for Replied Users -----------------

# Follow-up email body (Replace with relevant content)
follow_up_body = """
[Insert follow-up email content here]
"""

for receiver_email in merged_handles_df['Email IDs']:
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = "Re: Your Email Subject Here"
    msg.attach(MIMEText(follow_up_body, 'plain'))

    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, msg.as_string())
        server.quit()
        print(f"Follow-up email sent to {receiver_email}")
    except Exception as e:
        print(f"Failed to send follow-up email to {receiver_email}. Error: {e}")

print("Follow-up emails sent successfully!")